In [19]:
import pandas as pd
import yfinance as yf
import numpy as np
import pandas_ta as ta
import matplotlib.pyplot as plt
from matplotlib import rcParams
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# import ace_tools as tools; 

# 配置中文字体
plt.rcParams['font.family'] = 'Heiti TC'

# plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['figure.dpi'] = 150
plt.rcParams.update({'font.size': 12})

def read_excel(file_path):
    return pd.read_excel(file_path, sheet_name='Test')

def get_stock_data(stock_symbols, start_date, end_date):
    stock_data = {}
    for symbol in stock_symbols:
        stock_data[symbol] = yf.download(symbol, start=start_date, end=end_date)
    return stock_data

def get_stock_price(stock_symbols, start_date, end_date):
    stock_data = {}
    for symbol in stock_symbols:
        stock_data[symbol] = yf.download(symbol, start=start_date, end=end_date)['Adj Close']
    return pd.DataFrame(stock_data)

# 计算移动平均和标准差
def calculate_moving_average_and_std(stock_data, window):
    stock_data['Moving_Avg'] = stock_data['Adj Close'].rolling(window=window).mean()
    stock_data['Moving_Std'] = stock_data['Adj Close'].rolling(window=window).std()
    return stock_data

# Calculate technical indicators
def calculate_technical_indicators(stock_data):
    # 计算MACD
    macd = ta.macd(stock_data['Adj Close'])
    stock_data['MACD'] = macd['MACD_12_26_9']
    stock_data['MACD_signal'] = macd['MACDs_12_26_9']
    
    # 计算布林带
    bbands = ta.bbands(stock_data['Adj Close'])
    stock_data['BB_upper'] = bbands['BBU_5_2.0']
    stock_data['BB_middle'] = bbands['BBM_5_2.0']
    stock_data['BB_lower'] = bbands['BBL_5_2.0']
    
    return stock_data


# 生成交易信号
def generate_trading_signals(stock_data, window, threshold):
    stock_data = calculate_moving_average_and_std(stock_data, window)
    stock_data['Z_Score'] = (stock_data['Adj Close'] - stock_data['Moving_Avg']) / stock_data['Moving_Std']
    stock_data['Buy_Signal'] = np.where(stock_data['Z_Score'] < -threshold, 1, 0)
    stock_data['Sell_Signal'] = np.where(stock_data['Z_Score'] > threshold, 1, 0)
    return stock_data

from sklearn.ensemble import RandomForestClassifier

def generate_ml_trading_signals(stock_data, window, threshold):
    stock_data = calculate_moving_average_and_std(stock_data, window)
    stock_data['Z_Score'] = (stock_data['Adj Close'] - stock_data['Moving_Avg']) / stock_data['Moving_Std']
    
    # 计算其他技术指标
    stock_data = calculate_technical_indicators(stock_data)
    
    # 准备训练数据
    features = ['Z_Score', 'MACD', 'MACD_signal', 'BB_upper', 'BB_middle', 'BB_lower']
    stock_data.dropna(inplace=True)  # 移除缺失值
    X = stock_data[features]
    y = np.where(stock_data['Z_Score'] > threshold, 1, np.where(stock_data['Z_Score'] < -threshold, -1, 0))
    
    # 训练模型
    model = RandomForestClassifier()
    model.fit(X, y)
    
    # 生成交易信号
    stock_data['Predicted_Signal'] = model.predict(X)
    stock_data['Buy_Signal'] = np.where(stock_data['Predicted_Signal'] == -1, 1, 0)
    stock_data['Sell_Signal'] = np.where(stock_data['Predicted_Signal'] == 1, 1, 0)
    
    return stock_data


# Summarize results
def summarize_results(stock_symbols, stock_names, final_capitals, portfolio_values):
    results = []
    for symbol, name in zip(stock_symbols, stock_names):
        final_value = final_capitals[symbol]
        returns = np.diff(portfolio_values[symbol]) / portfolio_values[symbol][:-1]
        sharpe = np.mean(returns) / np.std(returns) * np.sqrt(252)  # Assuming daily returns, annualize Sharpe ratio
        drawdown = np.min(portfolio_values[symbol] / np.maximum.accumulate(portfolio_values[symbol])) - 1
        results.append([symbol, name, final_value, sharpe, returns.sum(), drawdown])
    
    summary_df = pd.DataFrame(results, columns=['Stock Symbol', 'Stock Name', 'Final Value', 'Sharpe Ratio', 'Total Return', 'Drawdown'])
    summary_df = summary_df.sort_values(by='Sharpe Ratio', ascending=False)
    return summary_df

# Visualize individual PnL and buy/sell points
def plot_individual_pnl_and_signals(stock_data, stock_symbols, stock_names, portfolio_values):
    for symbol, name in zip(stock_symbols, stock_names):
        fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1)

        # Plot portfolio value
        fig.add_trace(go.Scatter(x=stock_data[symbol].index, y=portfolio_values[symbol], mode='lines', name='Portfolio Value'), row=1, col=1)
        
        # Plot stock price with buy/sell signals
        fig.add_trace(go.Scatter(x=stock_data[symbol].index, y=stock_data[symbol]['Adj Close'], mode='lines', name='Stock Price'), row=2, col=1)
        fig.add_trace(go.Scatter(x=stock_data[symbol][stock_data[symbol]['Buy_Signal'] == 1].index, 
                                 y=stock_data[symbol]['Adj Close'][stock_data[symbol]['Buy_Signal'] == 1], 
                                 mode='markers', marker=dict(symbol='triangle-up', color='green', size=10), name='Buy Signal'), row=2, col=1)
        fig.add_trace(go.Scatter(x=stock_data[symbol][stock_data[symbol]['Sell_Signal'] == 1].index, 
                                 y=stock_data[symbol]['Adj Close'][stock_data[symbol]['Sell_Signal'] == 1], 
                                 mode='markers', marker=dict(symbol='triangle-down', color='red', size=10), name='Sell Signal'), row=2, col=1)
        
        fig.update_layout(title=f'{name} ({symbol})', xaxis_title='Date', yaxis_title='Value')
        fig.show()

def backtest_strategy(stock_data, initial_capital, transaction_cost, rebalance_period):
    capital = initial_capital
    position = 0
    cash = capital
    portfolio_value = []
    buy_price = 0
    days = 0

    for index, row in stock_data.iterrows():
        if days % rebalance_period == 0:
            # If there is a buy signal and no current position
            if row['Buy_Signal'] == 1 and position == 0:
                position = cash / (row['Adj Close'] * (1 + transaction_cost))
                buy_price = row['Adj Close']
                cash = 0
            # If there is a sell signal and we have a position
            elif row['Sell_Signal'] == 1 and position > 0:
                cash = position * (row['Adj Close'] * (1 - transaction_cost))
                position = 0
        # Calculate daily portfolio value
        daily_portfolio_value = cash + position * row['Adj Close']
        portfolio_value.append(daily_portfolio_value)
        days += 1

    # Final portfolio value
    final_value = portfolio_value[-1]
    return final_value, portfolio_value

# 网格搜索参数空间
param_grid = {
    'window': [10, 20, 30, 40],
    'threshold': [1.0, 1.5, 2.0, 2.5],
    'rebalance_period': [1, 3, 5, 7, 10],
}

# 定义评分函数
def score_function(params):
    window = params['window']
    threshold = params['threshold']
    rebalance_period = params['rebalance_period']
    
    total_final_capital = 0
    for symbol in stock_symbols:
        # stock_data[symbol] = generate_trading_signals(stock_data[symbol], window, threshold)
        stock_data[symbol] = generate_ml_trading_signals(stock_data[symbol], window, threshold)
        final_capital, _ = backtest_strategy(stock_data[symbol], initial_capital, transaction_cost, rebalance_period)
        total_final_capital += final_capital
    
    return total_final_capital

def plot_portfolio_values(portfolio_values, stock_symbols, stock_names):
    plt.figure(figsize=(14, 7))
    for symbol, name in zip(stock_symbols, stock_names):
        plt.plot(portfolio_values[symbol], label=name)
    plt.title('Portfolio Value Over Time')
    plt.xlabel('Days')
    plt.ylabel('Portfolio Value')
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1), ncol=1)
    plt.grid(True)
    plt.show()


# 读取Excel文件
df = pd.read_excel('Stocks.xlsx', sheet_name='Watchlist')

# 获取股票代码列表
stock_symbols = df['Stock Symbol'].tolist()
stock_names = df['Stock Name'].tolist()

# 定义时间范围
start_date = '2022-01-01'
end_date = '2024-08-05'

# 获取历史价格数据
stock_data = get_stock_data(stock_symbols, start_date, end_date)

# 设置参数
initial_capital = 100000
window = 10
threshold = 2.5
transaction_cost = 0.005  # 假设交易成本为0.5%
rebalance_period = 7  # 调仓周期为3天

# 获取上证指数数据
SSINDX = yf.download('000001.SS', start=start_date, end=end_date)
# 计算上证指数的PnL
SSINDX['Index_PnL'] = (SSINDX['Adj Close'] / SSINDX['Adj Close'].iloc[0]) * initial_capital

# 回测策略
final_capitals = {}
portfolio_values = {}

for symbol in stock_symbols:
    # 网格搜索
    # best_score = -np.inf
    # best_params = None

    # for window in param_grid['window']:
    #     for threshold in param_grid['threshold']:
    #         for rebalance_period in param_grid['rebalance_period']:
    #             params = {
    #                 'window': window,
    #                 'threshold': threshold,
    #                 'rebalance_period': rebalance_period,
    #             }
    #             score = score_function(params)
    #             if score > best_score:
    #                 best_score = score
    #                 best_params = params
    # print(f"{symbol} Best Parameters:", best_params)
    # window = best_params['window']
    # threshold = best_params['threshold']
    # rebalance_period = best_params['rebalance_period']
    
    stock_data[symbol] = generate_trading_signals(stock_data[symbol], window, threshold)
    # stock_data[symbol] = generate_ml_trading_signals(stock_data[symbol], window, threshold)
    final_capital, portfolio_value = backtest_strategy(stock_data[symbol], initial_capital, transaction_cost, rebalance_period)
    final_capitals[symbol] = final_capital
    portfolio_values[symbol] = portfolio_value

# 可视化回测结果
def plot_portfolio_values(portfolio_values, stock_symbols, stock_names, SSINDX):
    fig = make_subplots(rows=1, cols=1)
    
    
    for symbol, name in zip(stock_symbols, stock_names):
        fig.add_trace(go.Scatter(x=SSINDX.index, y=portfolio_values[symbol], mode='lines', name=name))
    
    # 添加上证指数作为参考值
    fig.add_trace(go.Scatter(x=SSINDX.index, y=SSINDX['Index_PnL'], mode='lines', name='上证指数'))

    fig.update_layout(title='PnL', xaxis_title='Date', yaxis_title='Value')
    fig.show()

plot_portfolio_values(portfolio_values, stock_symbols, stock_names, SSINDX)

# Summarize results
summary_df = summarize_results(stock_symbols, stock_names, final_capitals, portfolio_values)

# Visualize individual PnL and buy/sell points
# plot_individual_pnl_and_signals(stock_data, stock_symbols, stock_names, portfolio_values)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

/var/folders/2h/657ql8s954s35xlylnvyhmrw0000gn/T/ipykernel_78483/968668681.py:96: RuntimeWarning:

invalid value encountered in scalar divide



In [20]:
from scipy.optimize import minimize
def calculate_moving_average(stock_prices, short_window, long_window):
    short_ma = stock_prices.rolling(window=short_window).mean()
    long_ma = stock_prices.rolling(window=long_window).mean()
    return short_ma, long_ma

# Calculate Z-score for mean reversion
def calculate_z_score(stock_prices, window):
    rolling_mean = stock_prices.rolling(window=window).mean()
    rolling_std = stock_prices.rolling(window=window).std()
    z_score = (stock_prices - rolling_mean) / rolling_std
    return z_score

# Portfolio optimization
def portfolio_optimization(returns, cov_matrix):
    num_assets = len(returns)

    def objective(weights):
        return np.dot(weights.T, np.dot(cov_matrix, weights))
    
    def constraint(weights):
        return np.sum(weights) - 1

    cons = ({'type': 'eq', 'fun': constraint})
    bounds = tuple((0, 1) for _ in range(num_assets))
    result = minimize(objective, num_assets * [1. / num_assets,], bounds=bounds, constraints=cons)
    
    return result.x

def rebalance_portfolio(stock_prices, window, threshold, initial_capital, short_ma=None, long_ma=None):
    lot_size=100
    z_scores = calculate_z_score(stock_prices, window)
    cov_matrix = stock_prices.pct_change().cov()
    mean_returns = stock_prices.pct_change().mean()

    weights = portfolio_optimization(mean_returns, cov_matrix)
    portfolio_value = initial_capital
    portfolio_values = [initial_capital]
    positions = np.zeros(len(weights))
    cash = initial_capital
    buy_count = []
    sell_count = []
    annual_buy_count = 0
    annual_sell_count = 0
    current_year = stock_prices.index[window].year

    for date in stock_prices.index[window:]:
        if date.year != current_year:
            buy_count.append(annual_buy_count)
            sell_count.append(annual_sell_count)
            annual_buy_count = 0
            annual_sell_count = 0
            current_year = date.year
        
        # 根据Z-score和趋势调整权重
        new_weights = np.zeros(len(weights))
        for i, stock in enumerate(stock_prices.columns):
            if short_ma[stock].loc[date] > long_ma[stock].loc[date]:  # 确保在多头市场
                if z_scores[stock].loc[date] < -threshold:
                    new_weights[i] = weights[i] + 0.1
                elif z_scores[stock].loc[date] > threshold:
                    new_weights[i] = weights[i] - 0.1
                else:
                    new_weights[i] = weights[i]
            else:
                new_weights[i] = weights[i]  # 保持不变
        
        new_weights = np.clip(new_weights, 0, 1)
        new_weights = new_weights / np.sum(new_weights)
        
        # 根据新的权重进行买卖股票
        for i, stock in enumerate(stock_prices.columns):
            current_price = stock_prices[stock].loc[date]
            desired_position = portfolio_value * new_weights[i] // current_price
            if desired_position > positions[i]:
                buy_amount = (desired_position - positions[i]) // lot_size * lot_size
                if buy_amount > 0 and cash >= buy_amount * current_price * (1 + transaction_cost):
                    positions[i] += buy_amount
                    cash -= buy_amount * current_price * (1 + transaction_cost)
                    annual_buy_count += 1
            elif desired_position < positions[i]:
                sell_amount = (positions[i] - desired_position) // lot_size * lot_size
                if sell_amount > 0:
                    positions[i] -= sell_amount
                    cash += sell_amount * current_price * (1 - transaction_cost)
                    annual_sell_count += 1

        weights = new_weights
        portfolio_value = cash + np.sum(positions * stock_prices.loc[date])
        portfolio_values.append(portfolio_value)
    
    buy_count.append(annual_buy_count)
    sell_count.append(annual_sell_count)

    return portfolio_values, weights, buy_count, sell_count


# Calculate max drawdown
def calculate_max_drawdown(portfolio_values):
    cumulative_max = np.maximum.accumulate(portfolio_values)
    drawdown = (cumulative_max - portfolio_values) / cumulative_max
    max_drawdown = np.max(drawdown) * 100
    return max_drawdown

# Summarize PnL analysis
def pnl_summary(portfolio_values, dates, buy_count, sell_count):
    df = pd.DataFrame({'Date': dates, 'Portfolio Value': portfolio_values})
    df.set_index('Date', inplace=True)
    df['Return'] = df['Portfolio Value'].pct_change()
    df['Year'] = df.index.year

    summary = df.groupby('Year').agg({
        'Portfolio Value': ['first', 'last'],
        'Return': ['std']
    })
    summary.columns = ['Start Value', 'End Value', 'Return Std']
    summary['Annual Return %'] = ((summary['End Value'] / summary['Start Value']) - 1) * 100
    summary['Sharpe Ratio'] = summary['Annual Return %']  / (summary['Return Std'] * np.sqrt(252) * 100)
    
    # Calculate Max Drawdown per year
    max_drawdowns = []
    for year in summary.index:
        year_values = df[df['Year'] == year]['Portfolio Value']
        max_drawdowns.append(calculate_max_drawdown(year_values.values))

    summary['Max Drawdown %'] = max_drawdowns
    summary['Buy Count'] = buy_count
    summary['Sell Count'] = sell_count
    
    # Round to 2 decimal places
    summary = summary.round(2)

    return summary


# Summarize total PnL analysis
def pnl_summary_total(portfolio_values, dates):
    df = pd.DataFrame({'Date': dates, 'Portfolio Value': portfolio_values})
    df.set_index('Date', inplace=True)
    df['Return'] = df['Portfolio Value'].pct_change()

    total_return = ((df['Portfolio Value'][-1] / df['Portfolio Value'][0]) - 1) * 100
    sharpe_ratio = df['Return'].mean() / df['Return'].std() * np.sqrt(252)
    max_drawdown = calculate_max_drawdown(df['Portfolio Value'].values)

    summary_total = pd.DataFrame({
        'Total Return %': [total_return],
        'Sharpe Ratio': [sharpe_ratio],
        'Max Drawdown %': [max_drawdown]
    })

    # Round to 2 decimal places
    summary_total = summary_total.round(2)

    return summary_total


stock_data = get_stock_price(stock_symbols, start_date, end_date)

# Ensure stock_data is a DataFrame and handle any missing values
stock_prices = pd.DataFrame(stock_data).fillna(method='ffill').fillna(method='bfill')

short_window = 40
long_window = 200
short_ma, long_ma = calculate_moving_average(stock_prices, short_window, long_window)

# 调整rebalance_portfolio函数的调用
portfolio_values, final_weights, buy_count, sell_count = rebalance_portfolio(stock_prices, window, threshold, initial_capital, short_ma, long_ma)
print(f'Final Portfolio Value: {portfolio_values[-1]}')


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Final Portfolio Value: 120312.212166667


In [21]:
# PnL Summary
dates = stock_prices.index[window-1:]
summary = pnl_summary(portfolio_values, dates, buy_count, sell_count)
summary

,Start Value,End Value,Return Std,Annual Return %,Sharpe Ratio,Max Drawdown %,Buy Count,Sell Count
Year,,,,,,,,
2022,100000.00,107984.86,0.01,7.98,0.39,13.66,57,24
2023,109903.27,110245.20,0.01,0.31,0.02,17.14,65,33
2024,110645.86,120312.21,0.02,8.74,0.36,14.23,52,37


In [22]:
# Total PnL Summary
summary_total = pnl_summary_total(portfolio_values, dates)
summary_total

/var/folders/2h/657ql8s954s35xlylnvyhmrw0000gn/T/ipykernel_78483/354202215.py:142: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



,Total Return %,Sharpe Ratio,Max Drawdown %
0,20.31,0.48,18.85


In [23]:
# Interactive Plot for Portfolio Value
fig = go.Figure()
fig.add_trace(go.Scatter(x=dates, y=portfolio_values, mode='lines', name='Portfolio Value'))
fig.update_layout(title='Portfolio Value Over Time', xaxis_title='Date', yaxis_title='Portfolio Value')
fig.show()

In [24]:
summary_df

,Stock Symbol,Stock Name,Final Value,Sharpe Ratio,Total Return,Drawdown
12,600026.SS,中远海能,300911.655402,1.067064,1.494708,-0.462451
3,601898.SS,中煤能源,152106.618942,0.910943,0.473454,-0.175898
10,603799.SS,华友钴业,47580.437203,-0.946542,-0.648412,-0.589857
0,002299.SZ,圣农发展,100000.000000,NaN,0.000000,0.000000
1,600938.SS,中国海油,100000.000000,NaN,0.000000,0.000000
2,601225.SS,陕西煤业,100000.000000,NaN,0.000000,0.000000
4,603993.SS,洛阳钼业,100000.000000,NaN,0.000000,0.000000
5,600111.SS,北方稀土,100000.000000,NaN,0.000000,0.000000
6,000831.SZ,中国稀土,100000.000000,NaN,0.000000,0.000000
7,002460.SZ,赣锋锂业,100000.000000,NaN,0.000000,0.000000
